In [46]:
import angr
import claripy
from angr.sim_type import SimTypePointer, SimTypeFunction, SimTypeChar, SimTypeInt, parse_defns
from angr.errors import AngrCallableMultistateError

In [47]:
#angr logging is way too verbose
import logging
log_things = ["angr", "pyvex", "claripy", "cle"]
for log in log_things:
    logger = logging.getLogger(log)
    logger.disabled = True
    logger.propagate = False

In [48]:
charstar = SimTypePointer(SimTypeChar())
prototype = SimTypeFunction((charstar, charstar), SimTypeInt(False))

In [49]:
def loadproj(p, addr, find_addr, password):
    state = p.factory.call_state(addr, "userargs", password, prototype=prototype)
    simgr = p.factory.simgr(state)
    simgr.explore(find=(find_addr,), avoid=())
    return simgr

In [50]:
def getconcrete(p, addr, password):
    state = p.factory.call_state(addr, "userargs", password, prototype=prototype)
    simgr = p.factory.simgr(state)
    simgr.run()
    return simgr.deadended[0].regs.eax._model_concrete.value

In [51]:
#fauxware analysis

In [52]:
password_orig = claripy.BVS('pass_arg', 8*100)
pass_arg_orig = angr.PointerWrapper(password_orig, buffer=True)
p = angr.Project('fauxware', auto_load_libs=False)
simgr = loadproj(p, 0x401209, 0x40124a, pass_arg_orig)
p_state = simgr.found[0]

The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [53]:
#pmod fauxware with functionally identical strcmp

In [54]:
password_mod = claripy.BVS('pass_arg', 8*100)
pass_arg_mod = angr.PointerWrapper(password_mod, buffer=True)
p_mod = angr.Project('fauxware-mod', auto_load_libs=False)
simgr_mod = loadproj(p_mod, 0x4011e9, 0x401238, pass_arg_mod)
p_mod_state = simgr_mod.found[0]

The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [55]:
#different authentication than fauxware

In [56]:
password_diff = claripy.BVS('pass_arg', 8*100)
pass_arg_diff = angr.PointerWrapper(password_diff, buffer=True)
p_diff = angr.Project('fauxware-diff', auto_load_libs=False)
simgr_diff = loadproj(p_diff, 0x401209, 0x4012a3, pass_arg_diff)
p_diff_state = simgr_diff.found[0]

The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [57]:
#Analysis of equivalence of fauxware and itself

In [58]:
sol_orig_orig = claripy.Solver()
for i in p_state.solver.constraints:
    sol_orig_orig.add(i)
for i in p_state.solver.constraints:
    sol_orig_orig.add(i)
if sol_orig_orig.satisfiable(extra_constraints=(password_orig == password_orig,)):
    print("Same Function")
else:
    print("Diff Function")

Same Function


In [59]:
#Analysis between fauxware (auth_args) and fauxware-mod (auth_mod_args) should be functionally equiv

In [60]:
sol_orig_mod = claripy.Solver()
for i in p_state.solver.constraints:
    sol_orig_mod.add(i)
for i in p_mod_state.solver.constraints:
    sol_orig_mod.add(i)
if sol_orig_mod.satisfiable(extra_constraints=(password_orig == password_mod,)):
    print("Same Function")
else:
    print("Diff Function")

Same Function


In [61]:
#Analysis between fauxware (auth_args) and fauxware-diff (auth_diff_args)

In [67]:
sol_orig_diff = claripy.Solver()
for i in p_state.solver.constraints:
    sol_orig_diff.add(i)
for i in p_diff_state.solver.constraints:
    sol_orig_diff.add(i)
if sol_orig_diff.satisfiable(extra_constraints=(password_orig == password_diff,)):
    print("Same Function")
else:
    print("Diff Function")
    sol_orig_diff.add(password_orig != password_diff)
    a = [i.to_bytes(100, 'big') for i in sol_orig_diff.eval(password_orig,10)]
    a = list(set([i[0:i.find(b'\x00')] for i in a]))
    b = [i.to_bytes(100, 'big') for i in sol_orig_diff.eval(password_diff,10)]
    b = list(set([i[0:i.find(b'\x00')] for i in b]))
    print(a)
    print(b)
    

Diff Function
[b'SOSNEAKY']
[b'RNRMD@JX']
